# Combine LSOAs into larger polygons

This notebooks finds the collection of LSAOs surrounding each hospital that are closer to that hospital than any other, merges them into one shape, and then saves the new shape as geojson.

## Setup

In [1]:
import pandas as pd
import numpy as np

# For importing geojson:
import json

# For merging polygons:
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union

# For saving new geojson:
from geojson import FeatureCollection

# For drawing maps:
import folium
from folium import plugins

## Import data files

### Data table:

In [2]:
df_lsoa_hospitals = pd.read_csv('./lsoa_nearest_hospitals.csv')

df_lsoa_hospitals.head()

LSOA11CD       LSOA Nearest hospital postcode  \
0  E01031349  Adur 001A                    BN25BE   
1  E01031350  Adur 001B                   BN112DH   
2  E01031351  Adur 001C                   BN112DH   
3  E01031352  Adur 001D                   BN112DH   
4  E01031370  Adur 001E                   BN112DH   

                    Nearest hospital name  Time (mins) to nearest hospital  \
0  Royal Sussex County Hospital, Brighton                             17.6   
1                       Worthing Hospital                             18.7   
2                       Worthing Hospital                             17.6   
3                       Worthing Hospital                             17.6   
4                       Worthing Hospital                             16.5   

  Nearest MT hospital postcode                Nearest MT hospital name  \
0                       BN25BE  Royal Sussex County Hospital, Brighton   
1                       BN25BE  Royal Sussex County Hospital, Brighton   
2                       BN25BE  Royal Sussex County Hospital, Brighton   
3                       BN25BE  Royal Sussex County Hospital, Brighton   
4                       BN25BE  Royal Sussex County Hospital, Brighton   

   Time (mins) to nearest MT hospital  
0                                17.6  
1                                18.7  
2                                19.8  
3                                19.8  
4                                19.8

In [3]:
hospital_postcodes = sorted(list(set(df_lsoa_hospitals['Nearest hospital postcode'].values)))
hospital_mt_postcodes = sorted(list(set(df_lsoa_hospitals['Nearest MT hospital postcode'].values)))

### .geojson shape files:

In [4]:
with open('./LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3.geojson') as f:
    geojson_ew = json.load(f)

In [5]:
big_geojson_order = []

for i in range(len(geojson_ew['features'])):
    big_geojson_order.append(geojson_ew['features'][i]['properties']['LSOA11CD'])

## Make new shapes by combining LSOAs

In [6]:
def combine_polygons_for_hospital(
        hospital_postcode,
        df_lsoa_hospitals,
        geojson_ew,
        big_geojson_order,
        MT=False
        ):
    if MT == False:
        nearest_hospital_col = 'Nearest hospital postcode'
        MT_str = ''
    else:
        nearest_hospital_col = 'Nearest MT hospital postcode'
        MT_str = 'MT_'
    # Find all LSOAs that belong to this hospital:
    b = df_lsoa_hospitals[nearest_hospital_col] == hospital_postcode
    lsoa_names_here = df_lsoa_hospitals[b]['LSOA'].values
    lsoa_codes_here = df_lsoa_hospitals[b]['LSOA11CD'].values

    # Get the coordinates out of the big geojson:
    features_here = []
    for lsoa in lsoa_codes_here:
        # Find where it is in the big geojson:
        try:
            ind = big_geojson_order.index(lsoa)
            features_here.append(geojson_ew['features'][ind])
        except ValueError:
            print(f'Problem with {group}, LSOA {lsoa}')
            
    # Gather the polygons in here:
    polygons = []
    for i, feat in enumerate(features_here):
        # Get lists of coordinates, one list per separate polygon
        # in the LSOA.
        # (Have MultiPolygon e.g. when an LSOA on the mainland coastline
        # also contains an island, so there's a gap between areas.)
        lists_of_polygon_coordinates = feat['geometry']['coordinates']

        for list_of_coords in lists_of_polygon_coordinates:
            if feat['geometry']['type'] == 'MultiPolygon':
                # For MultiPolygons the coords are nested an extra time.
                list_of_coords = list_of_coords[0]
            
            # Create a new polygon from the coordinates:
            polygon = Polygon([(coords[0], coords[1]) for coords in list_of_coords])
            polygons.append(polygon)

    # Combine the polygons:
    combined_polygon = mapping(unary_union(polygons))
    
    # Give it any extra information to be saved in the geojson:
    combined_polygon['name'] = f'LSOAs_with_nearest_{MT_str}hospital_{hospital_postcode}'
    combined_polygon['LSOA11CD'] = ','.join(lsoa_codes_here)
    combined_polygon['LSOA11NM'] = ','.join(lsoa_names_here)
    combined_polygon['source'] = 'LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3'
    
    return combined_polygon

Nearest hospitals:

In [7]:
for hospital_postcode in hospital_postcodes:
    # Create the combined polygon:
    combined_polygon = combine_polygons_for_hospital(
        hospital_postcode,
        df_lsoa_hospitals,
        geojson_ew,
        big_geojson_order
        )

    # Save as a .geojson:
    save_name = 'lsoa_nearest_to_' + hospital_postcode + '.geojson'
    with open('./lsoa_nearest_hospital/'+save_name, 'w', encoding='utf-8') as f:
        json.dump(combined_polygon, f, ensure_ascii=False)

Nearest MT-providing hospitals:

In [8]:
for hospital_postcode in hospital_mt_postcodes:
    # Create the combined polygon:
    combined_polygon = combine_polygons_for_hospital(
        hospital_postcode,
        df_lsoa_hospitals,
        geojson_ew,
        big_geojson_order,
        MT=True
        )

    # Save as a .geojson:
    save_name = 'lsoa_nearest_MT_' + hospital_postcode + '.geojson'
    with open('./lsoa_nearest_hospital/'+save_name, 'w', encoding='utf-8') as f:
        json.dump(combined_polygon, f, ensure_ascii=False)

## Import and map the newly-made .geojson

In [9]:
# Create a map
nearest_hospital_map = folium.Map(location=[50.7, -4],
                        zoom_start=9,
                        tiles='cartodbpositron')

# Add region
files_to_draw = [
    'lsoa_nearest_to_EX25DW.geojson',
    'lsoa_nearest_to_EX314JB.geojson',
    'lsoa_nearest_to_PL68DH.geojson',
    'lsoa_nearest_to_TQ27AA.geojson',
    'lsoa_nearest_MT_PL68DH.geojson',
]
for file in files_to_draw:
    with open(f'./lsoa_nearest_hospital/'+file) as f:
        geojson_nearest_hospital = json.load(f)
    
    # Define style function for polygon colours:
    # (based on folium docs example but I haven't worked
    # out how to get the "if" statement working in the lambda
    # where the "if" depends on something directly related to "x".)
    if 'to_PL68DH' in file:
        style_function = lambda x: {
            "fillColor": 'red',
            'color': 'black',
            'fillOpacity':0.2
        }
    elif 'MT_PL68DH' in file:
        style_function = lambda x: {
            "fillColor": 'red',
            'color': 'none',
            'fillOpacity':0.2,
        }
    else:
        style_function = lambda x: {
            "fillColor": 'none',
            'color': 'black',
            # 'weight': 1,
            # 'dashArray': '5, 5',
            'fillOpacity':0.0
        }
    
    folium.GeoJson(
        geojson_nearest_hospital,
        name=hospital_postcode,
        style_function=style_function
        ).add_to(nearest_hospital_map)


# Generate map
nearest_hospital_map